In [31]:

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
# dbutils.widgets.text('incremental_flag', '0')


StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 33, Finished, Available, Finished)

In [23]:
# spark.sql("show tables in dbo").show()
# display(spark.sql("SHOW TABLES IN rprt"))

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 25, Finished, Available, Finished)

In [24]:
df = spark.read.parquet("Files/01. Bail/Bail_Landing_Layer/sap_hrp1001_2025-01-28.parquet")
display(df)

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, de86a698-25c8-4ed6-b334-84f416385532)

In [25]:
# spark.sql("select * from custodyrecord").show(10, False)

display(spark.sql("select * from custodyrecord LIMIT 10"))

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 591d927e-0960-4a7c-b33b-f1783e089d12)

In [26]:
# Create databases schemas
spark.sql("CREATE DATABASE IF NOT EXISTS bnx") #as bronze layer
spark.sql("CREATE DATABASE IF NOT EXISTS slv") #as silver layer
spark.sql("CREATE DATABASE IF NOT EXISTS gld") #as gold layer
spark.sql("CREATE DATABASE IF NOT EXISTS rprt") #as reporting layer

# Verify by listing databases
# spark.sql("SHOW DATABASES").show()
display(spark.sql("SHOW DATABASES"))

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f0db016e-b0cd-491d-bfb2-cd9cf2f8a7f4)

In [27]:
# create dim tables

d_bail_geo = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_d_bail_geo
(
             D_BAIL_GEO_DISTRICT_ID INT
			,D_BAIL_GEO_DISTRICT STRING
			,D_BAIL_GEO_AREA_ID INT
			,D_BAIL_GEO_AREA STRING
			,D_BAIL_GEO_POLICE_FORCE STRING
)"""
spark.sql(d_bail_geo)


d_date = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_d_date
(
        DateID INT,
        Date DATE,
        Quarter INT,
        QuarterName STRING,
        Month INT,
        MonthName STRING,
        Week INT,
        WeekName STRING,
        Day INT,
        DayName STRING,
        Year INT,
        YearMonth INT,
        YearMonthName STRING
)"""
spark.sql(d_date)


d_time = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_d_time
(
    TimeID INT,
    Hour24 INT,
    Hour12 INT,
    AmPmString STRING,
    MinuteShortString STRING,
    SecondShortString STRING,
    FullTimeString24 STRING,
    FullTimeString12 STRING,
    FullTime DATE,
    Shift STRING,
    Police_Hr INT,
    Time_Band STRING
)"""
spark.sql(d_time)


d_bail = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_d_bail
(
    D_BAIL_ID INT
      ,LoadID STRING
      ,D_BAIL_CUSTODY_ID INT
      ,D_BAIL_CUSTODY_REF STRING
      ,D_BAIL_TYPE STRING
      ,D_BAIL_TYPE_FULL STRING
      ,D_BAIL_CREATED_ON DATE
      ,D_BAIL_RELEASE_DATE DATE
      ,D_BAIL_RUI_FINALISE_DATE DATE
      ,D_BAIL_BAIL_EXPIRY_DATE DATE
      ,D_BAIL_BAIL_DATE DATE
      ,D_BAIL_TERMINATED_DATE DATE
      ,D_BAIL_END_DATE DATE
      ,D_BAIL_REVIEW_DUE_DATE DATE
      ,D_BAIL_LIVE_OR_HISTORIC STRING
      ,D_BAIL_STAGE STRING
      ,RN INT
      ,D_BAIL_BAIL_TO_RETURN_DATE DATE
      ,D_BAIL_SUSPENSION STRING
)"""
spark.sql(d_bail)


d_org = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_d_org
(
    PersonnelNumber STRING
    ,StartDate STRING
    ,EndDate STRING
    ,FullName STRING
    ,Section STRING
    ,SectionID STRING
    ,Unit STRING
    ,UnitID STRING
    ,Location STRING
    ,LocationID STRING
    ,Department STRING
    ,DepartmentID STRING
    ,Command STRING
    ,CommandID STRING
    ,Directorate STRING
    ,DirectorateID STRING
    ,Force STRING
    ,ForceID STRING
    ,Current INT
)"""
spark.sql(d_org)

# create fact tables

f_bail = f""" 
CREATE TABLE IF NOT EXISTS rprt.tbl_f_bail
(
    FK_D_BAIL_CUSTODY_ID INT
    ,LoadID INT
    ,FK_D_CREATE_DATE_ID INT
    ,FK_D_CREATE_TIME_ID INT
    ,FK_D_CUSTODY_REF STRING
    ,FK_D_BAIL_OIC_ID STRING
    ,F_BAIL_OIC_ID STRING
    ,FK_D_BAIL_GEO_DISTRICT_ID INT
    ,FK_D_BAIL_SUPERVISOR_ID STRING
    ,FK_D_BAIL_AUTHORISING_OFFICER_ID STRING
    ,F_MET_LENGTH_DAYS INT
    ,F_MET_PROXIMITY_EXPIRY INT
    ,BAIL_TYPE STRING
    ,LIVE_OR_HISTORIC STRING
    ,STAFF_MOVE_ID STRING
)"""
spark.sql(f_bail)

display(spark.sql("SHOW TABLES IN rprt"))

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, eb857668-6897-4f98-9e40-a02ddd06bf4f)

In [28]:
# tbl_d_time

# Create a tbl_d_time DataFrame with a range of numbers from 0 to 86399
numbers = spark.range(86400).selectExpr("id as Number")

# Then perform the time calculation and transformations
tbl_d_time = numbers.withColumn(
    "HOUR", (col("Number") / 3600).cast("int")
).withColumn(
    "MINUTE", ((col("Number") % 3600) / 60).cast("int")
).withColumn(
    "SECOND", (col("Number") % 60).cast("int")
).withColumn(
    "TimeID", (col("HOUR") * 10000 + col("MINUTE") * 100 + col("SECOND"))
).withColumn(
    "Hour24", col("HOUR")
).withColumn(
    "Hour12", (when(col("HOUR") % 12 == 0, 12).otherwise(col("HOUR") % 12)).cast("int")
).withColumn(
    "AmPmString", when(col("HOUR") < 12, "AM").otherwise("PM")
).withColumn(
    "MinuteShortString", format_string("%02d", col("MINUTE"))
).withColumn(
    "SecondShortString", format_string("%02d", col("SECOND"))
).withColumn(
    "FullTimeString24", concat_ws(":", format_string("%02d", col("HOUR")), format_string("%02d", col("MINUTE")), format_string("%02d", col("SECOND")))
).withColumn(
    "FullTimeString12", concat_ws(":", format_string("%02d", when(col("HOUR") % 12 == 0, 12).otherwise(col("HOUR") % 12)), format_string("%02d", col("MINUTE")), format_string("%02d", col("SECOND")), when(col("HOUR") < 12, lit("AM")).otherwise(lit("PM")))
).withColumn(
    "Shift", when((col("HOUR") > 5) & (col("HOUR") < 14), lit("EARLY"))
    .when((col("HOUR") >= 14) & (col("HOUR") < 22), lit("LATE"))
    .otherwise("NIGHT")
).withColumn(
    "Police_Hr", when(col("HOUR") > 5, col("HOUR") - 5).otherwise(col("HOUR") + 19)
).withColumn(
    "Time_Band", when((col("HOUR") >= 0) & (col("HOUR") < 3), "00:00-03:00")
    .when((col("HOUR") >= 3) & (col("HOUR") < 6), "03:00-06:00")
    .when((col("HOUR") >= 6) & (col("HOUR") < 9), "06:00-09:00")
    .when((col("HOUR") >= 9) & (col("HOUR") < 12), "09:00-12:00")
    .when((col("HOUR") >= 12) & (col("HOUR") < 15), "12:00-15:00")
    .when((col("HOUR") >= 15) & (col("HOUR") < 18), "15:00-18:00")
    .when((col("HOUR") >= 18) & (col("HOUR") < 21), "18:00-21:00")
    .when((col("HOUR") >= 21) & (col("HOUR") < 24), "21:00-00:00")
    .otherwise("Undefined")
)



StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d841517f-8826-45c4-98de-6a6d8ed3b40a)

In [29]:
tbl_d_time.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("rprt.tbl_d_time")
display(tbl_d_time.limit(10))

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 133ac53c-7063-4f4a-9f1b-9309d39953fd)

In [ ]:
# from pyspark.sql import *
# from pyspark.sql.functions import *
# from pyspark.sql.types import *

# Generate a range of dates starting from '1900-01-01'
dates_df = spark.range(0, (10 * 365))  # Approx. 10 years of days

# Add the 'Date' column starting from '1900-01-01'
dates_df = dates_df.withColumn("Date", date_add(lit("1900-01-01").cast(DateType()), col("id").cast("int")))

# Add variables for the Financial Year Start and Week Commencing
# Now, we should ensure that 'Date' column is properly referenced in the calculations
dates_with_variables_df = dates_df.withColumn(
    "FinancialYearStart", 
    date_format(
        date_add(
            col("Date"), 
            (months_between(col("Date"), lit("1900-02-01").cast(DateType())) * -1).cast("int")
        ), 
        'yyyy-02-01'
    )
).withColumn(
    "WeekCommencing", 
    date_sub(col("Date"), (dayofweek(col("Date")) - 1) % 7)
)

# Now we can continue to add more calculations
generated_dates_df = dates_with_variables_df.select(
    # DateID (YYYYMMDD format as INT)
    col("Date").cast("string").substr(1, 4).cast("int").alias("Year"),
    col("Date").cast("string").substr(5, 2).cast("int").alias("Month"),
    col("Date").cast("string").substr(7, 2).cast("int").alias("Day")
)

# Add other variables like Quarter, Week, DayName, etc.
generated_dates_df = generated_dates_df.withColumn(
    "Quarter", when(col("Month").between(1, 3), 1)
        .when(col("Month").between(4, 6), 2)
        .when(col("Month").between(7, 9), 3)
        .when(col("Month").between(10, 12), 4)
        .alias("Quarter")
)

generated_dates_df = generated_dates_df.withColumn(
    "QuarterName", concat(lit("Quarter - "), col("Quarter").cast("string"))
)

generated_dates_df = generated_dates_df.withColumn(
    "Week", weekofyear(col("Date"))
).withColumn(
    "WeekName", concat(lit("Week - "), col("Week").cast("string"))
).withColumn(
    "Day", dayofmonth(col("Date"))
).withColumn(
    "DayName", date_format(col("Date"), "EEEE")  # Weekday Name
)

generated_dates_df = generated_dates_df.withColumn(
    "YearMonth", concat(col("Year"), lit(""), col("Month"))
)

generated_dates_df = generated_dates_df.withColumn(
    "YearMonthName", concat(col("Year").cast("string"), lit(" - "), date_format(col("Date"), "MMMM"))
)

generated_dates_df.show(5)


In [ ]:
tbl_d_time.createOrReplaceTempView("tbl_d_time_temp")

spark.sql("""
    INSERT INTO tbl_d_time
    SELECT * FROM tbl_d_time_temp
""")


In [ ]:
df = spark.read.parquet("Files/01. Bail/Bail_Landing_Layer/sap_hrp1001_2025-01-28.parquet")
# df now is a Spark DataFrame containing parquet data from "Files/01. Bail/Bail_Landing_Layer/sap_hrp1001_2025-01-28.parquet".
display(df)

In [41]:
bail = f""" 
SELECT * FROM
		(
SELECT
			 CR.ID AS D_BAIL_ID
			,'LoadID' AS LoadID
			,CR.ID AS D_BAIL_CUSTODY_ID
			,CR.reference AS D_BAIL_CUSTODY_REF
			,CASE
				--Account for data excluded from dashboard
				WHEN CR.status = 'BAIL APPROVED' THEN 'DQ'
				WHEN CR.status = 'FINALISED'  AND COALESCE(CR.bailStartDate, CR.releaseDate) IS NULL THEN 'DQ'
				WHEN CR.status = 'BAILED' AND CR.releaseDate IS NOT NULL AND CR.bailStartDate IS NULL THEN 'CPS BAIL'
				
				--Data required in dashboard
				WHEN CR.finaliseDate <= COALESCE(CR.terminatedDate, '00:00:00') THEN 'BAIL' --COALESCE to 00:00:00 to handle comparison with null values
				WHEN CR.finaliseDate <= COALESCE(CR.expiredDate, '00:00:00') THEN 'BAIL'
				WHEN CR.expiredDate IS NOT NULL THEN 'RUI' --BAIL TO RUI
				WHEN CR.terminatedDate IS NOT NULL THEN 'RUI' --BAIL TO RUI
				WHEN CR.bailStartDate IS NOT NULL THEN 'BAIL'
				WHEN CR.releaseDate IS NULL THEN 'DQ'
				ELSE 'RUI'
				END AS D_BAIL_TYPE
			,CASE --Only difference from above is inclusion of 'BAIL to RUI' type for completeness - referred to as just RUI above
				WHEN CR.status = 'BAIL APPROVED' THEN 'DQ'
				WHEN CR.status = 'FINALISED'  AND COALESCE(CR.bailStartDate, CR.releaseDate) IS NULL THEN 'DQ'
				WHEN CR.status = 'BAILED' AND CR.releaseDate IS NOT NULL AND CR.bailStartDate IS NULL THEN 'CPS BAIL'
				
				WHEN CR.finaliseDate <= COALESCE(CR.terminatedDate, '00:00:00') THEN 'BAIL'
				WHEN CR.finaliseDate <= COALESCE(CR.expiredDate, '00:00:00') THEN 'BAIL'
				WHEN CR.expiredDate IS NOT NULL THEN 'BAIL TO RUI'
				WHEN CR.terminatedDate IS NOT NULL THEN 'BAIL TO RUI'
				WHEN CR.bailStartDate IS NOT NULL THEN 'BAIL'
				WHEN CR.releaseDate IS NULL THEN 'DQ'
				ELSE 'RUI'
				END AS D_BAIL_TYPE_FULL
			--,ARA.ACPO_GROUPING_GCV as D_BAIL_CRIME_TYPE 
			,CR.created as D_BAIL_CREATED_ON 
			,CR.releaseDate as D_BAIL_RELEASE_DATE
			,CR.finaliseDate as D_BAIL_RUI_FINALISE_DATE
			,CR.expiredDate as D_BAIL_BAIL_EXPIRY_DATE
			,CR.bailStartDate as D_BAIL_BAIL_DATE
			,CR.terminatedDate as D_BAIL_TERMINATED_DATE --Bail end date defined as minimum of terminated, finalise, expired and release
			,(SELECT MIN(enddate) FROM (VALUES (CR.terminatedDate), (CR.finaliseDate), 
				(CR.expiredDate),(CR.releaseDate)) AS value(enddate)) as D_BAIL_END_DATE
			,BR.reviewedOn as D_BAIL_REVIEW_DUE_DATE
			,CASE
				WHEN CR.status = 'FINALISED' THEN 'Historic'
				ELSE 'Live'
				END AS D_BAIL_LIVE_OR_HISTORIC
			,CR.status as D_BAIL_STAGE
			--,apr.update_date_time
			,ROW_NUMBER() OVER (partition by CR.id ORDER BY CR.created desc) as RN
			,CR.bailToReturnDate AS D_BAIL_BAIL_TO_RETURN_DATE
			,CASE
				WHEN SUSP.id = CR.id THEN 'Y'
				ELSE 'N'
				END AS D_BAIL_SUSPENSION

		FROM custodyRecord CR
		LEFT JOIN review BR ON CR.id = BR.custody_id AND BR.continueBail = 1 --Take record with most recent bail review date
				LEFT JOIN BailSuspension SUSP ON CR.id = SUSP.id
	) tbl_d_bail_01

WHERE tbl_d_bail_01.RN = 1
"""
spark.sql(bail).show(5)

StatementMeta(, 61925b8d-c122-4a9f-8b3f-9ccdc2fd8cc3, 43, Finished, Available, Finished)

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `review` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 61 pos 12;
'Project [*]
+- 'Filter ('tbl_d_bail_01.RN = 1)
   +- 'SubqueryAlias tbl_d_bail_01
      +- 'Project ['CR.ID AS D_BAIL_ID#4583, LoadID AS LoadID#4584, 'CR.ID AS D_BAIL_CUSTODY_ID#4585, 'CR.reference AS D_BAIL_CUSTODY_REF#4586, CASE WHEN ('CR.status = BAIL APPROVED) THEN DQ WHEN (('CR.status = FINALISED) AND isnull('COALESCE('CR.bailStartDate, 'CR.releaseDate))) THEN DQ WHEN ((('CR.status = BAILED) AND isnotnull('CR.releaseDate)) AND isnull('CR.bailStartDate)) THEN CPS BAIL WHEN ('CR.finaliseDate <= 'COALESCE('CR.terminatedDate, 00:00:00)) THEN BAIL WHEN ('CR.finaliseDate <= 'COALESCE('CR.expiredDate, 00:0